In [6]:
# from langchain.evaluation import load_evaluator
# from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# embedding_model = "thenlper/gte-large-zh"
# gpu = 0
# embeddings = HuggingFaceEmbeddings(model_name=embedding_model)#, model_kwargs={"device": f"cuda:{gpu}"


# """
# <EmbeddingDistance.COSINE: 'cosine'>,
#  <EmbeddingDistance.EUCLIDEAN: 'euclidean'>,
#  <EmbeddingDistance.MANHATTAN: 'manhattan'>,
#  <EmbeddingDistance.CHEBYSHEV: 'chebyshev'>,
#  <EmbeddingDistance.HAMMING: 'hamming'>

# """

# evaluator = load_evaluator("embedding_distance", embeddings=embeddings,distance_metric="cosine")#, distance_metric=EmbeddingDistance.EUCLIDEAN

# a1 = evaluator.evaluate_strings(prediction="社團與法人有什麼不同", reference="社團與法人差異有哪些")
# a2 = evaluator.evaluate_strings(prediction="社團與法人有什麼不同", reference="社團與法人相同點")
# a3 = evaluator.evaluate_strings(prediction="社團與法人有什麼不同", reference="社團與公司相同點")

# print(a1)
# print(a2)
# print(a3)
# """
# Note: This returns a distance score, meaning that the lower the number, 
# the more similar the prediction is to the reference, according to their embedded representation.
# """
# # print(_euclidean_relevance_score_fn(a1['score']))
# # print(_euclidean_relevance_score_fn(a2['score']))
# # print(_euclidean_relevance_score_fn(a3['score']))

# from langchain.evaluation import load_evaluator

# evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embeddings,distance_metric="cosine")
# evaluator.evaluate_string_pairs(
#     prediction="Seattle is hot in June", prediction_b="Seattle is cool in June."
# )
# evaluator.evaluate_string_pairs(
#     prediction="Seattle is warm in June", prediction_b="Seattle is cool in June."
# )

In [7]:
import evaluate
# !pip install sentencepiece

In [25]:
"""

Senetence Segmentation

"""


"""
ChrF and ChrF++ are two MT evaluation metrics. 
They both use the F-score statistic for character n-gram matches, 
and ChrF++ adds word n-grams as well which correlates more strongly with direct assessment. 
We use the implementation that is already present in sacrebleu. 
The implementation here is slightly different from sacrebleu in terms of the required input format. 
The length of the references and hypotheses lists need to be the same, 
so you may need to transpose your references compared to sacrebleu's required input format. 
"""

"""
ChrF and ChrF++ are two MT evaluation metrics that use the F-score statistic for character n-gram matches.
ChrF++ additionally includes word n-grams, which correlate more strongly with direct assessment. 
We use the implementation that is already present in sacrebleu.
While this metric is included in sacreBLEU, 
the implementation here is slightly different from sacreBLEU in terms of the required input format. 
Here, the length of the references and hypotheses lists need to be the same, 
so you may need to transpose your references compared to sacrebleu's required input format. 
"""

"""
char_order (int): Character n-gram order. Defaults to 6.
word_order (int): Word n-gram order. If equals to 2, the metric is referred to as chrF++. Defaults to 0.
beta (int): Determine the importance of recall w.r.t precision. Defaults to 2.
lowercase (bool): If True, enables case-insensitivity. Defaults to False.
whitespace (bool): If True, include whitespaces when extracting character n-grams. Defaults to False.
eps_smoothing (bool): If True, applies epsilon smoothing similar to reference chrF++.py, NLTK, and Moses implementations. If False, takes into account effective match order similar to sacreBLEU < 2.0.0. Defaults to False.

"""

"""Limitation
According to Popović 2017, 
chrF+ (where word_order=1) and chrF++ (where word_order=2) 
produce scores that correlate better with human judgements than chrF (where word_order=0) does.
"""
prediction = ["The relationship between cats and dogs is not exactly friendly.", " 好的書城 is just a genteel black hole that knows how to read."]
reference = ["貓和狗的relationship不怎麼friendly", "A good bookshop is just a genteel Black Hole that knows how to read."]
chrf = evaluate.load("chrf")
results = chrf.compute(predictions=prediction, references=reference,char_order=6,word_order=0,beta=1,lowercase=False,whitespace=False,eps_smoothing=False)
print(results)

"""
The chrF(++) score can be any value between 0.0 and 100.0, inclusive.
分數越高，表示質量越好。
"""

{'score': 55.68476903152963, 'char_order': 6, 'word_order': 0, 'beta': 1}


'\nThe chrF(++) score can be any value between 0.0 and 100.0, inclusive.\n分數越高，表示質量越好。\n'

In [11]:
######## 這不能使用toxicity
# male_model_completions = [
# 'was working so hard at an enterprise that he needed his own quarters',
# 'needs the answer',
# 'the pregnancy and the woman’s condition.'
# ]

# female_model_completions = [
# 'got up after 3 and gave him a few "fucks"',
# 'usually doesn’t have any money',
# 'the course and it would be a great opportunity to meet with patients during her time at this hospital.'
# ]
# toxicity = evaluate.load("toxicity", module_type="measurement")
# male_results = toxicity.compute(predictions=male_model_completions)

# female_results = toxicity.compute(predictions=female_model_completions)

# print(male_results)
# print(female_results)


In [9]:
"""
BERTScore leverages the pre-trained contextual embeddings from BERT and matches words 
in candidate and reference sentences by cosine similarity. 
It has been shown to correlate with human judgment on sentence-level and system-level evaluation. 
Moreover, BERTScore computes precision, recall, and F1 measure, which can be useful 
for evaluating different language generation tasks. 
"""

# 不用做segmentation
sentence1 = ["一隻貓稅在mattress上，cute cat"]
sentence2 = ["那有 cat 睡在墊子上，可怕的貓!!!"]
bertscore = evaluate.load("bertscore")
result = bertscore.compute(predictions=sentence1, references=sentence2, model_type = "bert-base-multilingual-cased",device=None,nthreads=4,rescale_with_baseline=False,) # (bertscore lang="en" "zh" "jp") #有中英夾雜使用jp 則是bert-base-multilingual-cased_L9_no-idf_version=0.3.
print(result)

{'precision': [0.68645840883255], 'recall': [0.6680428981781006], 'f1': [0.6771254539489746], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.41.2)'}


In [12]:
"""

Senetence Segmentation

"""

"""
FrugalScore is a reference-based metric for NLG models evaluation. 
It is based on a distillation approach that allows to learn a fixed,
low cost version of any expensive NLG metric, while retaining most of its original performance.
"""

"""
FrugalScore is a reference-based metric for Natural Language Generation (NLG) model evaluation. 
It is based on a distillation approach that allows to learn a fixed, 
low cost version of any expensive NLG metric, while retaining most of its original performance.
The FrugalScore models are obtained by continuing the pretraining of small models on a synthetic 
dataset constructed using summarization, backtranslation and denoising models. 
During the training, the small models learn the internal mapping of the expensive metric, 
including any similarity function.
"""

"""Limitation and Bias
FrugalScore is based on BertScore and MoverScore, 
and the models used are based on the original models used for these scores.
"""

prediction = ["The relationship between cats and dogs is not exactly friendly.", " 好的書城 is just a genteel black hole that knows how to read."]
reference = ["貓和狗的relationship不怎麼friendly", "A good bookshop is just a genteel Black Hole that knows how to read."]
frugalscore = evaluate.load("frugalscore") #evaluate.load("frugalscore", "moussaKam/frugalscore_medium_bert-base_mover-score")
results = frugalscore.compute(predictions=prediction, references=reference, batch_size=16, max_length=64,device=None)
print(results)
# 分數越高越好

c:\Users\foresight_User\anaconda3\lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'scores': [0.436601, 0.8199912]}


In [14]:
# pip install mauve-text
"""

Senetence Segmentation

"""
"""
MAUVE is a measure of the gap between neural text and human text. 
It is computed using the Kullback–Leibler (KL) divergences between the two distributions of text in a quantized embedding 
space of a large language model. 
MAUVE can identify differences in quality arising from model sizes and decoding algorithms.
"""

"""

The metric takes two lists of strings of tokens separated by spaces: 
one representing predictions (i.e. the text generated by the model) 
 the second representing references (a reference text for each prediction)


mauve: MAUVE score, which ranges between 0 and 1. Larger values indicate that P and Q are closer.

The original MAUVE paper reported values ranging from 0.88 to 0.94 for 
open-ended text generation using a text completion task in the web text domain. 
The authors found that bigger models resulted in higher MAUVE scores and that MAUVE is correlated with human judgments.


frontier_integral: Frontier Integral, which ranges between 0 and 1. Smaller values indicate that P and Q are closer.


"""

mauve = evaluate.load('mauve')
predictions = ["The relationship between cats and dogs is not exactly friendly.", " 好的書城 is just a genteel black hole that knows how to read."]
references = ["貓和狗的relationship不怎麼friendly", "A good bookshop is just a genteel Black Hole that knows how to read."]
mauve_results = mauve.compute(predictions=predictions, references=references)

print(mauve_results)

Loading tokenizer
Tokenizing text...
Loading tokenizer
Loading model
Featurizing tokens


Featurizing p:   0%|          | 0/2 [00:00<?, ?it/s]

Tokenizing text...
Featurizing tokens


Featurizing q:   0%|          | 0/2 [00:00<?, ?it/s]

seed = 25
performing clustering in lower dimension = 1
kmeans time: 0.33 s
total discretization time: 0.43 seconds
namespace(p_hist=array([1., 0.]), q_hist=array([0.5, 0.5]), divergence_curve=array([[1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 3.12501563e-02],
       [9.95665182e-01, 3.83262005e-02],
       [9.82728863e-01, 4.66296573e-02],
       [9.61393611e-01, 5.63137024e-02],
       [9.31995061e-01, 6.75437565e-02],
       [8.94998923e-01, 8.04979556e-02],
       [8.50996750e-01, 9.53676224e-02],
       [8.00700392e-01, 1.12357737e-01],
       [7.44935073e-01, 1.31687407e-01],
       [6.84630987e-01, 1.53590342e-01],
       [6.20813269e-01, 1.78315319e-01],
       [5.54590183e-01, 2.06126657e-01],
       [4.87139290e-01, 2.37304688e-01],
       [4.19691287e-01, 2.72146226e-01],
       [3.53511139e-01, 3.10965041e-01],
       [2.89875920e-01, 3.54092326e-01],
       [2.30048606e-01, 4.01877170e-01],
       [1.75246688e-01, 4.54687031e-01],
       [1.26603877e-01, 5.129

In [36]:
# perplexity 不可使用，主要評估訓練模型後的好壞
"""
This metric outputs a dictionary with the perplexity scores for the text input in the list, 
and the average perplexity. 
If one of the input texts is longer than the max input length of the model, 
then it is truncated to the max length for the perplexity computation.
"""



'\nThis metric outputs a dictionary with the perplexity scores for the text input in the list, \nand the average perplexity. \nIf one of the input texts is longer than the max input length of the model, \nthen it is truncated to the max length for the perplexity computation.\n'

In [15]:
"""

Word Segmentation

"""

"""
ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, 
is a set of metrics and a software package used for evaluating automatic summarization 
and machine translation software in natural language processing. 
The metrics compare an automatically produced summary or translation 
against a reference or a set of references (human-produced) summary or translation.
"""

"""

"rouge1": unigram (1-gram) based scoring
"rouge2": bigram (2-gram) based scoring
"rougeL": Longest common subsequence based scoring.
"rougeLSum": splits text using "\n"

"""
# 要做segmentation
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=['你好啊', 'general kenobi'],
                         references=['hello 你好啊', 'kenobi將軍'])#, tokenizer=lambda x: x.split()
print(results)
# UNIGEN 用 'rougeL'

{'rouge1': 0.3333333333333333, 'rouge2': 0.0, 'rougeL': 0.3333333333333333, 'rougeLsum': 0.3333333333333333}


In [16]:
"""

Word Segmentation

"""



"""
BLEU (Bilingual Evaluation Understudy) is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another. 
Quality is considered to be the correspondence between a machine's output and that of a human: 
"the closer a machine translation is to a professional human translation, the better it is" – this is the central idea behind BLEU. 
BLEU was one of the first metrics to claim a high correlation with human judgements of quality, 
and remains one of the most popular automated and inexpensive metrics.

Scores are calculated for individual translated segments—generally sentences—by comparing them with a set of good quality reference translations. 
Those scores are then averaged over the whole corpus to reach an estimate of the translation's overall quality. 
Neither intelligibility nor grammatical correctness are not taken into account.
"""

"""
max_order (int): Maximum n-gram order to use when computing BLEU score. Defaults to 4.
"""

"""
BLEU's output is always a number between 0 and 1. 
This value indicates how similar the candidate text is to the reference texts,
with values closer to 1 representing more similar texts. 
Few human translations will attain a score of 1, 
since this would indicate that the candidate is identical to one of the reference translations. 
For this reason, it is not necessary to attain a score of 1. 
Because there are more opportunities to match, adding additional reference translations will increase the BLEU score.
"""
predictions = ['你好啊', 'general kenobi']
references = [['hello there', 'kenobi將軍'],
                ["你好啊", "general 肯諾比"]]
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references,  max_order=1)#, 用 1 grame
print(results)

{'bleu': 0.3333333333333333, 'precisions': [0.3333333333333333], 'brevity_penalty': 1.0, 'length_ratio': 1.5, 'translation_length': 3, 'reference_length': 2}


In [24]:
"""

Word Segmentation

"""

"""
SacreBLEU provides hassle-free computation of shareable, comparable, and reproducible BLEU scores. 
Inspired by Rico Sennrich's `multi-bleu-detok.perl`, 
it produces the official WMT scores but works with plain text. 
It also knows all the standard test sets and handles downloading, processing, and tokenization for you.
"""

"""
SacreBLEU provides hassle-free computation of shareable, comparable, and reproducible BLEU scores. 
Inspired by Rico Sennrich's multi-bleu-detok.perl, 
it produces the official Workshop on Machine Translation (WMT) 
scores but works with plain text. 
It also knows all the standard test sets and handles downloading, processing, and tokenization.
"""

"""
tokenize (str): 
Tokenization method to use for BLEU. 
If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for Japanese and '13a' (mteval) otherwise. Possible values are:
   'none': No tokenization.
   'zh': Chinese tokenization.
   '13a': mimics the mteval-v13a script from Moses.
   'intl': International tokenization, mimics the mteval-v14 script from Moses
   'char': Language-agnostic character-level tokenization.
   'ja-mecab': Japanese tokenization. Uses the MeCab tokenizer.

   intl 方法適合：大多數印歐語系語言（如英語、法語、德語等），這些語言的單詞邊界明確，使用國際標準的單詞級分詞更能反映翻譯質量。
   char 方法適合：漢藏語系語言（如中文、日語、韓語等），這些語言單詞邊界不明顯，字符級分詞更能準確評估翻譯質量。
   !! 這部分需要好好review

   intl (International tokenization)：
    基於國際標準的分詞方法，模仿 Moses mteval-v14 腳本。
    主要針對單詞級別的匹配，考慮單詞邊界、標點符號和其他語言特性。
    適合處理大多數語言，特別是那些詞彙邊界清晰的語言，如英語、法語等。
   char (Character-level tokenization)：
    基於字符級別的分詞方法，不依賴於單詞邊界。
    將文本分解為單個字符進行匹配。
    更加語言無關，適合處理那些單詞邊界不明顯或形態複雜的語言，如中文、日語等。

lowercase (bool): 
   If True, lowercases the input, enabling case-insensitivity. Defaults to False.
force (bool): 
   If True, insists that your tokenized input is actually detokenized. Defaults to False.
use_effective_order (bool): 
   If True, stops including n-gram orders for which precision is 0. 
   This should be True, if sentence-level BLEU will be computed. Defaults to False.

   use_effective_order 參數決定了在計算 BLEU 分數時是否忽略精確度為 0 的 n-gram 順序。
   設為 True 時，有助於提高句子級別 BLEU 分數的準確性，特別適用於短句子的評估。
   設為 False 時，則適合評估長文本的整體翻譯質量
"""

"""Limitation and Bias
Because what this metric calculates is BLEU scores, 
it has the same limitations as that metric, except that sacreBLEU is more easily reproducible.

"""
predictions = ['你好啊', 'general kenobi']
references = [['hello there', 'kenobi將軍'],
                ["你好啊", "general kenobi"]]
sacrebleu = evaluate.load("sacrebleu")
results = sacrebleu.compute(predictions=predictions, 
                            references=references,tokenize="char",lowercase=True,use_effective_order=False) # 要調整tokenize，為何用char
print(results)

{'score': 65.39657130693087, 'counts': [13, 12, 11, 10], 'totals': [16, 14, 12, 10], 'precisions': [81.25, 85.71428571428571, 91.66666666666667, 100.0], 'bp': 0.7316156289466418, 'sys_len': 16, 'ref_len': 21}


In [25]:
# ! pip install sacremoses
sari = evaluate.load("sari")
sources=["'kenobi將軍，您好!"]
predictions = ['你好啊! general kenobi']
references = [['hello there! kenobi將軍',
                "你好啊! general 肯諾比"]]
sari_score = sari.compute(sources=sources, predictions=predictions, references=references)
"""

SARI (system output against references and against the input sentence) is 
a metric used for evaluating automatic text simplification systems.
The metric compares the predicted simplified sentences against the reference and the source sentences. 
It explicitly measures the goodness of words that are added, deleted and kept by the system.

The range of values for the SARI score is between 0 and 100 -- 
the higher the value, the better the performance of the model being evaluated,
with a SARI of 100 being a perfect score.

"""

In [19]:
"""

Sentence Segmentation

"""

"""
The metric can take on any value 0 and above. 0 is a perfect score, 
meaning the predictions exactly match the references and no edits were necessary.
 Higher scores are worse. 
 Scores above 100 mean that the cumulative number of edits, num_edits, 
 is higher than the cumulative length of the references, ref_length.

"""
predictions = ["這句話對嗎??",
               "那這句話呢?",
          "這笑話很難笑"     ]
references = [["這句話對嗎", "這句話對嗎!?!"],
               ["這句話怎麼樣?", "wHaT aBoUt ThIs SeNtEnCe?"],
               ["你的笑話...", "...TERrible"]]

ter = evaluate.load("ter")
results = ter.compute(predictions=predictions,
                        references=references,
                         case_sensitive=True,support_zh_ja_chars=True,ignore_punct=True,normalized=True)


results

{'score': 72.0, 'num_edits': 9, 'ref_length': 12.5}

In [20]:
"""

Word Segmentation

"""

"""
Word error rate (WER) is a common metric of the performance of an automatic speech recognition (ASR) system.

The general difficulty of measuring the performance of ASR systems lies in the fact that the recognized word sequence can have a different 
length from the reference word sequence (supposedly the correct one). The WER is derived from the Levenshtein distance, working at the word level.

This problem is solved by first aligning the recognized word sequence with the reference (spoken) 
word sequence using dynamic string alignment. 
Examination of this issue is seen through a theory called the power law that states the correlation between 
perplexity and word error rate (see this article for further information).

"""

"""
The lower the value, the better the performance of the ASR system, with a WER of 0 being a perfect score.

"""

# !pip install jiwer
wer = evaluate.load("wer")
predictions = ['你好啊', 'general kenobi']
references1 = ['hello there', 'kenobi將軍']
references2 = ["你好啊", "general 肯諾比"]
wer_score1 = wer.compute(predictions=predictions, references=references1)
wer_score2 = wer.compute(predictions=predictions, references=references2)

print(wer_score1)
print(wer_score2)


1.3333333333333333
0.3333333333333333


In [21]:
"""

Word Segmentation

"""

"""
https://huggingface.co/datasets/google/xtreme_s

wer: 
Word error rate (WER) is a common metric of the performance of an automatic speech recognition system. 
The lower the value, the better the performance of the ASR system, with a WER of 0 being a perfect score (see WER score for more information). 
It is returned for the mls, fleurs-asr, voxpopuli and babel subsets of the benchmark.

cer: 
Character error rate (CER) is similar to WER, but operates on character instead of word. 
The lower the CER value, the better the performance of the ASR system, 
with a CER of 0 being a perfect score (see CER score for more information). 
It is returned for the mls, fleurs-asr, voxpopuli and babel subsets of the benchmark.

多語言語音基準 (Multilingual Speech Benchmark, MLS) 
是一個用於評估多語言自動語音識別 (ASR) 模型性能的數據集。
MLS 數據集包含多種語言的語音數據，旨在促進多語言語音技術的發展。評估模型性能時，
常用的指標包括字錯誤率 (WER) 和語音片段的準確性。


Fleurs-ASR：
這代表的是 Fleurs 的自動語音識別（Automatic Speech Recognition, ASR）任務。
Fleurs 是一個多語種語音數據集，
用於訓練和評估語音識別系統。在這個任務中，模型需要將語音轉錄為相應的文本，並在多種語言上進行評估。
"""

xtreme_s_metric = evaluate.load('xtreme_s', 'mls')
predictions = ['你好啊', 'general kenobi']
references = ['hello there', 'kenobi將軍']
results = xtreme_s_metric.compute(predictions=predictions, references=references)
print(results)


{'wer': 1.3333333333333333, 'cer': 1.105263157894737}


In [5]:
""""ArithmeticError
pip uninstall spacy
pip uninstall srsly
pip uninstall blis
pip uninstall cymem
pip uninstall preshed
pip uninstall murmurhash

pip install thinc==8.2.2
pip install blis==0.7.8
pip install cymem
pip install preshed
pip install murmurhash
pip install spacy

python -m spacy download en_core_web_sm
python -m spacy download ja_core_news_sm
python -m spacy download de_core_news_sm
python -m spacy download es_core_news_sm
python -m spacy download it_core_news_sm
python -m spacy download ko_core_news_sm
python -m spacy download ru_core_news_sm
python -m spacy download fr_core_news_sm

"""